<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/Controlmeme_Colab_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#How to use ControlMeme generation backend

1. Run all cells until you see a meme.koll.ai link at the bottom of the page (this can take a while)

<img src="https://raw.githubusercontent.com/koll-ai/control-meme-api/main/images/run_all.png"  height="70">

2. Click the link to start creating new memes !

<img src="https://user-images.githubusercontent.com/22277706/224489141-42e03196-7f56-4ae1-93a8-184886a659c7.png"  height="70">


#1. Install dependencies ( ~2-3 min)

Source: https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb

In [1]:
#@markdown # Install/Update AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [2]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui
  !git checkout a9fed7c
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet
  !git checkout 8682c1e 
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf gcolab.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  
  !npm install -g localtunnel
  !pip install flask flask_cors

clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [3]:
#@markdown # Model Download/Load
Model_Version = "1.5" #@param [ "1.5", "v1.5 Inpainting ", "V2.1-512px", "V2.1-768px"]

Redownload_the_original_model = False #@param {type:"boolean"}

def rmv():
  !wget -q -O /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/precompiled/README.md
  !mv /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
  time.sleep(2)
  !rm /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/trashfile.f
  clear_output()

if Redownload_the_original_model:
  with capture.capture_output() as cap:
    rmv()

#@markdown Or
Path_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full path of your trained model or to a folder containing multiple models.

#@markdown Or
MODEL_LINK = "https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.ckpt" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}
Use_temp_storage = False #@param {type:"boolean"}



def newmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-v1-5"
      if os.path.exists('/content/stable-diffusion-v1-5/unet/diffusion_pytorch_model.bin'):
        !wget -q -O stable-diffusion-v1-5/vae/diffusion_pytorch_model.bin https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.bin
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-v1-5"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V1.5','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-v1-5'):
      !rm -r /content/stable-diffusion-v1-5

    return model

def V2():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
      %cd /content/
      clear_output()
      !mkdir "/content/stable-diffusion-V2"
      %cd "/content/stable-diffusion-V2"
      !git init
      !git lfs install --system --skip-repo
      if Model_Version == "V2.1-768px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1"
      elif Model_Version == "V2.1-512px":
        !git remote add -f origin  "https://huggingface.co/stabilityai/stable-diffusion-2-1-base"
      !git config core.sparsecheckout true
      !echo -e "scheduler\ntext_encoder\ntokenizer\nunet\nvae\nfeature_extractor\nmodel_index.json\n!*.safetensors" > .git/info/sparse-checkout
      !git pull origin fp16
      %cd /content
      !wget -O convertosdv2.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosdv2.py
      clear_output()
      !python /content/convertosdv2.py --fp16 /content/stable-diffusion-V2 /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
      !rm /content/convertosdv2.py
      if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'):
        model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
        clear_output()
        inf('\u2714 Done','success', '50px')
      else:
        inf('\u2718 Something went wrong, try again','danger', "250px")

    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt'
      clear_output()
      inf('\u2714 Model already exists, check the box "Redownload_the_original_model" to redownload/download the V2','primary', '700px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git
      !rm -r /content/convertosdv2.py
    if os.path.exists('/content/stable-diffusion-V2'):
      !rm -r /content/stable-diffusion-V2

    return model

def inpmdl():

    if not os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
      %cd /content/
      clear_output()
      !git init
      !git lfs install --system --skip-repo
      !$fgitclone --branch fp16 "https://huggingface.co/runwayml/stable-diffusion-inpainting"
      if os.path.exists('/content/stable-diffusion-inpainting'):
        !$fgitclone "https://huggingface.co/stabilityai/sd-vae-ft-mse"
        !rm -r /content/stable-diffusion-inpainting/vae
        !mv /content/sd-vae-ft-mse /content/stable-diffusion-inpainting/vae        
        !wget -O convertosd.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
        !sed -i '201s@.*@    model_path = "/content/stable-diffusion-inpainting"@' /content/convertosd.py
        !sed -i '202s@.*@    checkpoint_path= "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt"@' /content/convertosd.py
        clear_output()       
        !python /content/convertosd.py
        !rm /content/convertosd.py
        if os.path.exists('/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'):
          model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
          clear_output()
          inf('\u2714 Done','success', '50px')
        else:
          inf('\u2718 Something went wrong, try again','danger', "250px")
      else:
        inf('\u2718 Something went wrong','danger', "200px")


    else:
      model='/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/sd-v1-5-inpainting.ckpt'
      clear_output()
      inf('\u2714 inpainting model already exists','primary', '250px')

    if os.path.exists('/content/.git'):
      !rm -r /content/.git

    if os.path.exists('/content/stable-diffusion-inpainting'):
      !rm -r /content/stable-diffusion-inpainting

    return model

if (Path_to_MODEL !=''):
  if os.path.exists(str(Path_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(Path_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        Path_to_MODEL=input()
      if os.path.exists(str(Path_to_MODEL)):
        inf('\u2714 Using the trained model.','success', '200px')

  model=Path_to_MODEL

elif MODEL_LINK != "":
  gdrv="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
  tmp="/content"
  pth=tmp if Use_temp_storage else gdrv
  %cd $pth
  clear_output()
  if not safetensors:
    modelname="model.ckpt"
  else:
    modelname="model.safetensors"
  !gdown --fuzzy -O $modelname $MODEL_LINK
  if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
      model=f'{pth}/{modelname}'
      clear_output()
      inf('\u2714 Model downloaded, using the trained model.','success', '350px')
  else:
    if Use_temp_storage:
      !rm $pth/$modelname
    else:
      rmv()
    inf('\u2718 Wrong link, check that the link is valid','danger', "300px")


elif Model_Version=="1.5":
  model=newmdl()

elif Model_Version=="V2.1-512px" or Model_Version=="V2.1-768px":
  model=V2()

else:
   model=inpmdl()

Button(button_style='success', description='✔ Model downloaded, using the trained model.', disabled=True, layo…

In [4]:
#@markdown # ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse

Model = "All" #@param [ "All", "Canny", "Depth", "HED", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "T2iadapter_Models"]

#@markdown - Download/update ControlNet extension and its models.

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
Depth='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
HED='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
MLSD='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
Normal='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
OpenPose='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
Scribble='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
Seg='https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'


with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git pull
    %cd /content

!cp /content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models/*.yaml /content/gdrive/MyDrive/sd/stable-diffusion-webui/models
mdldir="/content/gdrive/MyDrive/sd/stable-diffusion-webui/extensions/sd-webui-controlnet/models"

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt
with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()
!rm CN_models.txt

if Model == "All": 
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

else:
  download(globals()[Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [5]:
#@markdown # Start Stable-Diffusion

# fix versions
# %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui
# !git checkout 13081dd45ece33457f6cb2cad3a8e7840a0a6eaf --force


from IPython.utils import capture
import time
import sys
import fileinput
import os
Use_localtunnel = False

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.9/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.9/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !nohup python -u /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --nowebui --no-hashing	--api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers --disable-console-progressbars > /content/output_webui.txt &
  else:
    !nohup python -u /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --nowebui --no-hashing --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers --disable-console-progressbars > /content/output_webui.txt &
except:
   !nohup python -u /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --nowebui --no-hashing --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars > /content/output_webui.txt &

nohup: redirecting stderr to stdout


In [6]:
#@markdown # Define Helper function

import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request, send_file
import base64
from io import BytesIO
import os

import subprocess
import time
from flask_cors import CORS

import requests


from io import BytesIO
import sys
import cv2
import imutils
from PIL import Image
import os
import subprocess
import time

from base64 import b64encode
import logging
import requests
import base64
import cv2
import sys
logger = logging.getLogger(__name__)


def cv_to_b64(cv_image):
    # im_arr: image in Numpy one-dim array format.
    _, im_arr = cv2.imencode('.jpg', cv_image)
    im_bytes = im_arr.tobytes()
    im_b64 = base64.b64encode(im_bytes)
    return im_b64


def adjust_size(width, height):
    ratio = width/height
    if (ratio > 1):
        width = 512
        round_width = width

        height = round(512 * 1/ratio)
        round_height = ((height + 32) // 64) * 64
        round_height = int(max(round_height, 64))
    else:
        height = 512
        round_height = height

        width = round(512 * ratio)
        round_width = ((width + 32) // 64) * 64
        round_width = int(max(round_width, 64))

    return round_width, round_height


class controlnetRequest():
    def __init__(self, host_url):
        self.host_url = host_url

    def send_request(self, url, body):
        print('sending request')
        print(body)
        r = requests.post(url, json=body, headers={
                          'Bypass-Tunnel-Reminder': 'please'})
        resp = r.json()
        return resp

    def send_txt2img_request(self, input_image, module, model, **kwargs):
        """ Send a request txt2img to the controlnet api
        Args:
            hint_image: the hint image
            kwargs: the parameters of the controlnet
                - nb_steps: the number of steps
                - prompt: the prompt
                - negative_prompt: the negative prompt
                - seed: the seed
                - subseed: the subseed
                - subseed_strength: the subseed strength
                - batch_size: the batch size
                - n_iter: the number of iterations
                - steps: the number of steps
                - cfg_scale: the cfg scale
                - restore_faces: true if we want to restore faces
                - eta: the eta
                - sampler_index: the sampler index
                - controlnet_module: the controlnet module
                - controlnet_model: the controlnet model
                - controlnet_guidance: the controlnet guidance

        """
        url = f"{self.host_url}/controlnet/txt2img"
        width, height = adjust_size(input_image.shape[1], input_image.shape[0])

        input_image_b64 = cv_to_b64(input_image).decode("utf-8")

        # get the parameters from the kwargs or use default value
        body = {
            "prompt": kwargs.get("prompt", ""),
            "negative_prompt": kwargs.get("negative_prompt", ""),
            "seed": kwargs.get("seed", -1),
            "subseed": kwargs.get("subseed", -1),
            "subseed_strength": kwargs.get("subseed_strength", 0),
            "batch_size": kwargs.get("batch_size", 1),
            "n_iter": kwargs.get("n_iter", 1),
            "steps": kwargs.get("steps", 30),
            "cfg_scale": kwargs.get("cfg_scale", 7),
            "width": width,
            "height": height,
            "restore_faces": kwargs.get("restore_faces", True),
            "eta": kwargs.get("eta", 0),
            "sampler_index": kwargs.get("sampler_index", "Euler a"),
            "controlnet_input_image": [input_image_b64],
            "controlnet_module": module,
            "controlnet_model": model,
            "controlnet_guidance": kwargs.get("controlnet_guidance", 1.0),
            'controlnet_threshold_a': kwargs.get('controlnet_threshold_a', 100),
            'controlnet_threshold_b': kwargs.get('controlnet_threshold_b', 100),
            'controlnet_preprocessor_res': kwargs.get('controlnet_preprocessor_res', 64),
        }
        print('before send:')
        print(body)

        return self.send_request(url, body)

    def send_detect_request(self, input_image, module, **kwargs):
        """ Send a request detect to the controlnet api
        Args:
            input_image: the input image 
            module: the controlnet module [canny, depth, ...]
            kwargs: the parameters of the controlnet
                - preprocessor_res: the preprocessor resolution
                - threshold_a: the threshold a
                - threshold_b: the threshold b
        """
        url = f"{self.host_url}/controlnet/detect"

        hint_b64 = cv_to_b64(input_image).decode("utf-8")

        body = {
            'controlnet_module': module,
            'controlnet_preprocessor_res': kwargs.get('controlnet_preprocessor_res', 64),
            'controlnet_input_images': [hint_b64],
            'conytolnet_threshold_a': kwargs.get('controlnet_threshold_a', 100),
            'controlnet_threshold_b': kwargs.get('controlnet_threshold_b', 100),
        }

        return self.send_request(url, body)


def generate_controlnet(input_image, module, model, host_url="http://localhost:7861", kwargs={}):

    req = controlnetRequest(host_url)
    resp = req.send_txt2img_request(input_image, module, model, **kwargs)

    return resp['images']


def generate_hint(input_image, module, host_url="http://localhost:7861", kwargs = {}):
    req = controlnetRequest(host_url)
    resp = req.send_detect_request(input_image, module, **kwargs)
    print(resp)
    return resp['images']

def write_b64_img(path, cv_image):
    decoded_data = base64.b64decode(cv_image)
    np_data = np.fromstring(decoded_data,np.uint8)
    img = cv2.imdecode(np_data,cv2.IMREAD_UNCHANGED)
    cv2.imwrite(path, img)


In [ ]:
#@markdown # 2. Run this cell and click on the link below to be redirected to the website
#@markdown You have to wait for A1111 API to be started before starting to generate images  (logs in /content/output_webui.txt)  (~1 min)
#@markdown * Ready when 127.0.0.1 url is displayed at the end of the file
#@markdown * Click again on the file to refresh the output


# run localtunnel
with open('/content/output_tunnel.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)


# os.environ['COMMANDLINE_ARGS'] = "--port 7861 --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers" 
# os.environ['COMMANDLINE_ARGS'] = "--no-hashing --nowebui --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt {model_path} --xformers" 
# os.environ['REQS_FILE'] = "requirements.txt"

# with open('/content/output_webui.txt', 'w') as f:
#     proc_automatic = subprocess.Popen(['python', '-u', 'launch.py'], stdout=f, stderr=f)

time.sleep(3)

app = Flask(__name__)
CORS(app)

params = dict()

API_URL  = ""
BACK_URL = "https://controle-meme-back-n7bkntouja-uc.a.run.app"

#Route to generate the hint according to processor, thresholdA and thresholdB
@app.route('/hint/', methods=['POST'])
def hint():
    params = request.get_json()
    print(params)
    meme_url = params['controlnet_basememe_url']
    processor_module = params['controlnet_module']
    thresholdA = params['controlnet_threshold_a']
    thresholdB = params['controlnet_threshold_b']
    
    # read the image from the url to cv2
    base_meme = imutils.url_to_image(meme_url)   

    hint = generate_hint(base_meme, processor_module, kwargs=params)
    hint = hint[0]
    

    
    
    return hint


@app.route('/', methods=['POST'])
def index():
    global params

    params = request.get_json()
    meme_url = params['controlnet_basememe_url']
    module = params['controlnet_module']
    model  = params['controlnet_model']
    
    # read the image from the url to cv2
    base_meme = imutils.url_to_image(meme_url)    

    # todo: send request to sdwebui api to get the generated image
    try: 
        last_image = generate_controlnet(base_meme, module, model, kwargs=params)
    except requests.exceptions.ConnectionError:
        return "Automatic api subprocess is not fully started yet, please try again in a minute", 500
      
    last_image_b64 = last_image[0]
    last_hint_b64 = last_image[1]
    
    # with open('./last_meme.jpeg', 'w') as f:
    #   Image.open(BytesIO(base64.b64decode(last_image))).save(f, format="JPEG")
    # with open("./last_meme.jpeg", "wb") as f:
    #    f.write(last_image_b64)


    write_b64_img('./last_meme.jpeg', last_image_b64)
    write_b64_img('./last_hint.jpeg', last_hint_b64)



    return last_image_b64



@app.route('/save_last/')
def save_last():
    global params

    # last_image = Image.open('./last_meme.jpeg')
    # buffered = BytesIO()
    # last_image.save(buffered, format="JPEG")
    # img_str = base64.b64encode(buffered.getvalue())

    last_image = cv2.imread('./last_meme.jpeg')
    img_str = cv_to_b64(last_image)

    last_hint = cv2.imread('./last_hint.jpeg')
    hint_str = cv_to_b64(last_hint)

    data = dict(
        imageb64=img_str.decode("utf-8"),
        hintb64=hint_str.decode("utf-8"),
        **params
    )
    url = f'{BACK_URL}/api/save_variation/'
    requests.post(url, json = data)

    return "ok"


@app.route('/generate_video/', methods=['POST'])
def generate_video():
    params = request.get_json()
    
    parent_url = params['parent_url']
    variation_url = params['variation_url']

    path_video = generate_video_ffmep(parent_url, variation_url)

    return send_file(path_video)


@app.route('/hello/')
def hello():    
    return "hello"

# todo: add route to check if automatic api is started


if __name__ == '__main__':
  with open('/content/output_tunnel.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1].strip()

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'>>>>  https://meme.koll.ai?gpuURL={url}  <<<<')

from IPython.utils import capture

# with capture.capture_output() as cap:
#   app.run()

app.run()